In [3]:
import zipfile
from pathlib import Path

zip_path = Path("data/Face_mask_detection.zip")
extract_path = Path("data/Face_mask_detection.zip")

extract_path.mkdir(parents=True, exist_ok=True)

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted to:", extract_path)

IsADirectoryError: [Errno 21] Is a directory: 'data/Face_mask_detection.zip'

In [ ]:
import os


os.listdir("data") # Asks the OS: “What files/folders exist directly inside data/?”


['Face_mask_detection', 'Face_mask_detection.zip']